# Continuous Stirred-Tank Reactor

Simulating the startup transient of an exothermic first-order reaction in a cooled CSTR, showing the dynamic interaction between concentration decay and temperature rise.

The CSTR model couples a material balance with an energy balance. For a first-order irreversible reaction $A \to \text{products}$ with Arrhenius kinetics:

$$\frac{dC_A}{dt} = \frac{C_{A,\text{in}} - C_A}{\tau} - k(T)\, C_A$$

$$\frac{dT}{dt} = \frac{T_\text{in} - T}{\tau} + \frac{(-\Delta H_\text{rxn})}{\rho\, C_p}\, k(T)\, C_A + \frac{UA}{\rho\, C_p\, V}\,(T_c - T)$$

where $k(T) = k_0 \exp(-E_a / RT)$ and $\tau = V/F$ is the residence time.

In [ ]:
import matplotlib.pyplot as plt

from pathsim import Simulation, Connection
from pathsim.blocks import Source, Scope

from pathsim_chem.process import CSTR

Configure the reactor with parameters typical of an exothermic liquid-phase reaction. The reactor starts empty ($C_A = 0$) at ambient temperature and is fed with a concentrated stream.

In [ ]:
cstr = CSTR(
    V=1.0,            # reactor volume [m³]
    F=0.05,           # volumetric flow rate [m³/s] -> tau = 20 s
    k0=1e6,           # pre-exponential factor [1/s]
    Ea=40000.0,       # activation energy [J/mol]
    n=1.0,            # first-order reaction
    dH_rxn=-40000.0,  # exothermic [J/mol]
    rho=1000.0,       # density [kg/m³]
    Cp=4184.0,        # heat capacity [J/(kg·K)]
    UA=800.0,         # cooling jacket [W/K]
    C_A0=0.0,         # start empty
    T0=300.0,         # initial temperature [K]
)

Feed a constant concentration of 1000 mol/m³ at 320 K, with the coolant held at 290 K. A `Scope` records both the outlet concentration and temperature.

In [ ]:
# Constant feed and coolant conditions
C_feed = Source(func=lambda t: 1000.0)   # feed concentration [mol/m³]
T_feed = Source(func=lambda t: 320.0)    # feed temperature [K]
T_cool = Source(func=lambda t: 290.0)    # coolant temperature [K]

scp = Scope(labels=["C_A [mol/m³]", "T [K]"])

sim = Simulation(
    blocks=[C_feed, T_feed, T_cool, cstr, scp],
    connections=[
        Connection(C_feed, cstr),          # C_in -> port 0
        Connection(T_feed, cstr[1]),        # T_in -> port 1
        Connection(T_cool, cstr[2]),        # T_c  -> port 2
        Connection(cstr, scp),              # C_out -> scope port 0
        Connection(cstr[1], scp[1]),        # T_out -> scope port 1
    ],
    dt=0.1,
)

sim.run(200)

In [ ]:
time, signals = scp.read()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(time, signals[0])
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Concentration [mol/m³]")
ax1.set_title("Outlet Concentration")
ax1.grid(True, alpha=0.3)

ax2.plot(time, signals[1])
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Temperature [K]")
ax2.set_title("Reactor Temperature")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

The reactor starts cold and empty. As fresh feed enters, concentration rises initially but then the Arrhenius kinetics kick in — the exothermic reaction heats the fluid, which accelerates the rate, consuming more reactant. The cooling jacket prevents thermal runaway and the system settles to a steady state where reaction rate balances the feed.